In [1]:
import sys
import folium
import geopandas as gpd

# path to folder to CityGeoTools
folder = "/Users/lizzy/Documents/ИТМО/Прикладные методы анализа ГД/Online/CityGeoTools-master"
sys.path.append(folder)

from metrics.data import CityInformationModel as BaseModel

/Users/lizzy/Documents/ИТМО/Прикладные методы анализа ГД/Online/venv/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
pip install mapclassify

Note: you may need to restart the kernel to use updated packages.


In [3]:
import matplotlib.pyplot as plt
from mapclassify import classify
from shapely.geometry import Point
from folium.plugins import MarkerCluster
from branca.colormap import LinearColormap

In [4]:
city_model = BaseModel.CityInformationModel(city_name="Miass", city_crs=32641, cwd="../")
city_model.update_layer("Services", "/Users/lizzy/Documents/ИТМО/Прикладные методы анализа ГД/Online/Services.geojson")

Validation of Services layer...
Services layer loaded successfully!


In [5]:
city_model.methods.services_clusterization.message

{'Services': 'Layer matches specification'}

In [6]:
# Мы решили выбрать  сервисы, которые пользуются популярностью в выходные
# Соотвественно это кафе и фаст-фуд, торговые центры, МФЦ и спортзалы

In [7]:
from metrics.calculations import services_clusterization

service_clusteriztion = services_clusterization.ServicesClusterization(city_model).get_clusters_polygon(
    service_types=['mall', 'fast_food', 'cafe', 'mfz', 'fitness'], n_std=2, condition_value=1000
    )

service_poins = gpd.GeoDataFrame.from_features(service_clusteriztion['services']).set_crs(4326)
service_clusters = gpd.GeoDataFrame.from_features(service_clusteriztion['polygons']).set_crs(4326)

service_clusters.head(10)

,geometry,cafe,fast_food,fitness,mall,mfz
0,"POLYGON ((60.14584 55.13165, 60.14445 55.13180...",0.71,0.14,0.00,0.14,0.00
1,"POLYGON ((60.15278 55.14016, 60.15189 55.14046...",0.00,0.67,0.00,0.33,0.00
2,"POLYGON ((60.14445 55.14200, 60.14351 55.14238...",0.50,0.25,0.12,0.00,0.12
3,"POLYGON ((60.15512 55.16150, 60.15435 55.16221...",0.67,0.33,0.00,0.00,0.00
4,"POLYGON ((60.15587 55.15308, 60.15446 55.15306...",0.60,0.30,0.00,0.10,0.00
5,"POLYGON ((60.09961 55.00625, 60.09836 55.00672...",1.00,0.00,0.00,0.00,0.00
6,"POLYGON ((60.10055 54.98217, 60.10057 54.98221...",0.50,0.50,0.00,0.00,0.00
7,"POLYGON ((60.10634 54.98541, 60.10545 54.98488...",0.60,0.20,0.00,0.00,0.20
8,"POLYGON ((60.09616 55.07438, 60.09619 55.07434...",0.50,0.50,0.00,0.00,0.00
9,"POLYGON ((60.13241 55.09922, 60.13091 55.09936...",0.67,0.33,0.00,0.00,0.00


In [8]:
service_clusters.explore()

In [9]:
service_poins.head(10)

,geometry,service_code,administrative_unit_id,municipality_id,block_id,id,cluster
0,POINT (60.14429 55.14244),cafe,1,1,116,0,3
1,POINT (60.10803 55.05530),cafe,1,1,369,1,16
2,POINT (60.15203 55.15673),cafe,1,1,106,2,6
3,POINT (60.09286 55.04568),cafe,1,1,410,3,28
4,POINT (60.10611 55.05693),cafe,1,1,403,4,16
5,POINT (60.09544 55.07420),cafe,1,1,508,7,11
6,POINT (60.10860 55.05105),cafe,1,1,371,8,17
7,POINT (60.10878 55.05987),cafe,1,1,365,9,18
8,POINT (60.10786 55.04597),cafe,1,1,372,10,26
9,POINT (60.15547 55.15687),cafe,1,1,98,11,6


In [10]:
service_poins.explore()

In [11]:
points = service_poins
polygons = service_clusters

Удаляем одиночные кластеры

In [12]:
attributes_to_check = ['cafe', 'fast_food', 'fitness', 'mall', 'mfz']
polygons = polygons[~((polygons[attributes_to_check] == 1).any(axis=1))]

In [13]:
m = folium.Map(location=[points.geometry.y.mean(), points.geometry.x.mean()], zoom_start=10)


for idx, row in polygons.iterrows():
    popup_text = f"Cafe: {row['cafe']} <br>Fast food: {row['fast_food']} <br>Fitness center: {row['fitness']} <br>Mall: {row['mall']} <br>MFZ: {row['mfz']}"
    folium.GeoJson(row['geometry'], popup=folium.Popup(popup_text)).add_to(m)

marker_cluster = MarkerCluster().add_to(m)
for idx, row in points.iterrows():
    popup_text = f"Cluster: {row['cluster']}<br>Service type: {row['service_code']}"
    folium.Marker([row['geometry'].y, row['geometry'].x], popup=folium.Popup(popup_text)).add_to(marker_cluster)


m.save("clusters_with_popup.html")

In [14]:
polygons_gdf = polygons

houses_gdf = gpd.read_file('/Users/lizzy/Documents/ИТМО/Прикладные методы анализа ГД/Online/building2.geojson')

polygons_gdf['residents_count'] = 0

# Проходим по каждому полигону и каждому дому, проверяя, попадает ли дом в полигон
for poly_index, poly_row in polygons_gdf.iterrows():
    for house_index, house_row in houses_gdf.iterrows():
        if poly_row['geometry'].contains(house_row['geometry']):
            polygons_gdf.loc[poly_index, 'residents_count'] += house_row['population']



In [15]:
m = folium.Map(location=[polygons_gdf['geometry'].centroid.y.mean(), polygons_gdf['geometry'].centroid.x.mean()],
               zoom_start=12)

colormap = LinearColormap(colors=['yellow', 'orange', 'red'],
                           vmin=polygons_gdf['residents_count'].min(),
                           vmax=polygons_gdf['residents_count'].max())

folium.GeoJson(
    polygons_gdf,
    name='Residents Count',
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['residents_count']),
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7
    },
    highlight_function=lambda x: {'weight': 2, 'color': 'black'},
    smooth_factor=2.0,
    tooltip=folium.GeoJsonTooltip(fields=['residents_count'],
                                   aliases=['Residents Count'],
                                   labels=True,
                                   sticky=True)
).add_to(m)

colormap.add_to(m)

m.save('clusters_residents.html')